In [1]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# Install Tensorflow from the website: https://www.tensorflow.org/versions/r0.12/get_started/os_setup.html

# Installing Keras
# 


# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [3]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''
#import torch
#import torch.nn as nn
#from torch.autograd import Variable
#import torch.nn.functional as F
#from torch import optim

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop


from keras import backend as K
from keras import layers
import numpy as np
import tensorflow as tf

class PlasticDense(layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        #self.batch_size = batch_size
        super(PlasticDense, self).__init__(**kwargs)
        
    def build(self, input_shape):
        
        #Define set of traditional weights
        self.w = self.add_weight(name='w', 
                                 shape=(input_shape[1], self.output_dim),
                                 initializer='uniform',
                                 trainable=True)
        
        #Define our plasticity coefficient
        self.alpha = self.add_weight(name='alpha', 
                              shape=(1, 1),
                              initializer='uniform',
                              trainable=True)
        
        
        #The Hebbian trace
        self.hebb = self.add_weight(name='hebb', 
                              shape=(input_shape[1], self.output_dim),
                              initializer='zeros',
                              trainable=False)
        
        #Step size will be optimized
        self.eta = self.add_weight(name='eta', 
                                      shape=(1, 1),
                                      initializer='uniform',
                                      trainable=False)
        super(PlasticDense, self).build(input_shape)

        
        #yout = F.tanh( yin.mm(self.w + torch.mul(self.alpha, hebb)) + input )
        #hebb = (1 - self.eta) * hebb + self.eta * torch.bmm(yin.unsqueeze(2), yout.unsqueeze(1))[0] # bmm here is used to implement an outer product between yin and yout, with the help of unsqueeze (i.e. added empty dimensions)
        #return yout, hebb

    def call(self, x):
        
        #X (layer input)     : shape(?, INPUT_DIM)
        #W                   : shape(INPUT_DIM, OUTPUT_DIM)
        #hebb                : shape(INPUT_DIM, OUTPUT_DIM)
        #Y (layer output)    : shape(?, OUTPUT_DIM)
        #ETA                 : scalar (one per layer)
        
        #yout = K.maximum(0.0, np.add((K.dot(self.y, np.add(K.dot(self.alpha, K.transpose(self.hebb)), self.w))), x))
        #hebb = (1 - 0.01) * self.hebb + 0.01 * K.dot(self.y, yout)
        #yout = K.maximum(0.0, np.add(self.y * np.add(self.alpha * self.hebb, self.w), x))
        
        #y = K.dot(x, self.w)
        #y = F.tanh( yin.mm(self.w + torch.mul(self.alpha, hebb)) + yin )
        #plastic_y = self.alpha * (K.dot(x, self.hebb)) 
        #self.hebb = (1 - self.eta) * self.hebb + self.eta * torch.bmm(yin.unsqueeze(2), yout.unsqueeze(1))[0]
        #model_out = K.maximum(0.0, y + plastic_y)
        
        #Hebbian update - option 1
        #self.hebb = self.eta * K.dot(x, model_out) + (1 - self.eta) * self.hebb
        #print(self.hebb)
        
        #Hebbian update - option 2
        #self.hebb +=self.eta * K.dot(model_out, (x - (K.dot(model_out, self.hebb))))
        
        y = K.dot(x, self.w)
        plastic_y = self.alpha * (K.dot(x, self.hebb))   
        model_out = K.maximum(0.0, y + plastic_y)
        
        #Hebbian update - option 1
        self.hebb = self.eta * K.dot(x, model_out) + (1 - self.eta) * self.hebb
        print(self.hebb)

        return model_out
        #return K.maximum(0.0, y)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


In [ ]:
def testClassic(num_neurons, num_layers, epochs):
    print("Classic: n: {} l: {} e: {}".format(num_neurons, num_layers, epochs))
    classifier = Sequential()
    classifier.add(Dense(output_dim = num_neurons, init = 'uniform', activation = 'relu', input_dim = 11))
    for l in range(num_layers - 1):
        classifier.add(Dense(output_dim = num_neurons, init = 'uniform', activation = 'relu'))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    train_history = classifier.fit(X_train, y_train, batch_size = 128, nb_epoch = epochs)
    
    loss_history = train_history.history["loss"]
    np.savetxt("results/n_{}_l_{}_e_{}_loss_classic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(loss_history), delimiter=",")
    
    acc_history = train_history.history["acc"]
    np.savetxt("results/n_{}_l_{}_e_{}_acc_classic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(acc_history), delimiter=",")
    # TODO Append test prediction results to file or to its own file
    y_pred = classifier.predict(X_test)
    return (y_pred > 0.5)


def testPlastic(num_neurons, num_layers, epochs):
    print("Plastic: n: {} l: {} e: {}".format(num_neurons, num_layers, epochs))
    classifier = Sequential()
    classifier.add(PlasticDense(num_neurons, input_shape=(11,)))
    for l in range(num_layers - 1):
        classifier.add(PlasticDense(num_neurons))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    train_history = classifier.fit(X_train, y_train, batch_size = 128, nb_epoch = epochs)
        
    loss_history = train_history.history["loss"]
    np.savetxt("results/n_{}_l_{}_e_{}_loss_plastic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(loss_history), delimiter=",")
    
    acc_history = train_history.history["acc"]
    np.savetxt("results/n_{}_l_{}_e_{}_acc_plastic.txt".format(num_neurons, num_layers, epochs), 
                  np.array(acc_history), delimiter=",")
    # TODO Append test prediction results to file or to its own file
    y_pred = classifier.predict(X_test)
    return (y_pred > 0.5)
    
    
def extensive_test():
    neuron_combos = [4, 6, 8, 12, 16]
    layer_combos = [2, 4, 8, 16]
    epochs = 200
    
    i = 0
    for n in neuron_combos:
        for l in layer_combos:
            testClassic(n, l, epochs)
            testPlastic(n, l, epochs)
            i += 1
            print("Try: {}/{}".format(i, len(neuron_combos)*len(layer_combos)))

extensive_test()
#y_pred = testClassic(4, 4, 2)
#y_pred = testPlastic(4, 4, 20)

Classic: n: 4 l: 2 e: 200
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", input_dim=11, units=4)`
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=4, activation="relu", kernel_initializer="uniform")`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", kernel_initializer="uniform")`
  import sys
/home/mun/.local/lib/python2.7/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/200
8000/8000 [==============================] - 1s 86us/step - loss: 0.6845 - acc: 0.7864
Epoch 2/200
8000/8000 [==============================] - 0s 28us/step - loss: 0.6672 - acc: 0.7960
Epoch 3/200
8000/8000 [==============================] - 0s 30us/step - loss: 0.6513 - acc: 0.7960
Epoch 4/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.6368 - acc: 0.7960
Epoch 5/200
8000/8000 [==============================] - 0s 27us/step - loss: 0.6234 - acc: 0.7960
Epoch 6/200
8000/8000 [==============================] - 0s 36us/step - loss: 0.6112 - acc: 0.7960
Epoch 7/200
8000/8000 [==============================] - 0s 27us/step - loss: 0.6001 - acc: 0.7960
Epoch 8/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5900 - acc: 0.7960
Epoch 9/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5808 - acc: 0.7960
Epoch 10/200
8000/8000 [==============================] - 0s 27us/step - loss: 0.5724 - acc: 0.7960
Epoch 11/

8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 84/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 85/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 86/200
8000/8000 [==============================] - 0s 27us/step - loss: 0.5059 - acc: 0.7960
Epoch 87/200
8000/8000 [==============================] - 0s 28us/step - loss: 0.5059 - acc: 0.7960
Epoch 88/200
8000/8000 [==============================] - 0s 28us/step - loss: 0.5059 - acc: 0.7960
Epoch 89/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 90/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 91/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 92/200
8000/8000 [==============================] - 0s 28us/step - loss: 0.5059 - acc: 0.7960
Epoch 93/200


8000/8000 [==============================] - 0s 28us/step - loss: 0.5059 - acc: 0.7960
Epoch 165/200
8000/8000 [==============================] - 0s 29us/step - loss: 0.5059 - acc: 0.7960
Epoch 166/200
8000/8000 [==============================] - 0s 29us/step - loss: 0.5059 - acc: 0.7960
Epoch 167/200
8000/8000 [==============================] - 0s 28us/step - loss: 0.5059 - acc: 0.7960
Epoch 168/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 169/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 170/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 171/200
8000/8000 [==============================] - 0s 28us/step - loss: 0.5059 - acc: 0.7960
Epoch 172/200
8000/8000 [==============================] - 0s 26us/step - loss: 0.5059 - acc: 0.7960
Epoch 173/200
8000/8000 [==============================] - 0s 27us/step - loss: 0.5059 - acc: 0.7960
Epoc

...

8000/8000 [==============================] - 1s 77us/step - loss: 0.3857 - acc: 0.7960
Epoch 10/200
8000/8000 [==============================] - 1s 80us/step - loss: 0.3820 - acc: 0.7960
Epoch 11/200
8000/8000 [==============================] - 1s 79us/step - loss: 0.3777 - acc: 0.7960
Epoch 12/200
8000/8000 [==============================] - 1s 86us/step - loss: 0.3742 - acc: 0.8456
Epoch 13/200
8000/8000 [==============================] - 1s 79us/step - loss: 0.3711 - acc: 0.8530
Epoch 14/200
8000/8000 [==============================] - 1s 87us/step - loss: 0.3686 - acc: 0.8540
Epoch 15/200
8000/8000 [==============================] - 1s 100us/step - loss: 0.3647 - acc: 0.8572
Epoch 16/200
8000/8000 [==============================] - 1s 94us/step - loss: 0.3622 - acc: 0.8551
Epoch 17/200
8000/8000 [==============================] - 1s 92us/step - loss: 0.3614 - acc: 0.8565
Epoch 18/200
8000/8000 [==============================] - 1s 101us/step - loss: 0.3589 - acc: 0.8578
Epoch 19/20

8000/8000 [==============================] - 1s 81us/step - loss: 0.3308 - acc: 0.8659
Epoch 92/200
8000/8000 [==============================] - 1s 92us/step - loss: 0.3326 - acc: 0.8635
Epoch 93/200
8000/8000 [==============================] - 1s 79us/step - loss: 0.3335 - acc: 0.8658
Epoch 94/200
8000/8000 [==============================] - 1s 81us/step - loss: 0.3311 - acc: 0.8656
Epoch 95/200
8000/8000 [==============================] - 1s 76us/step - loss: 0.3300 - acc: 0.8668
Epoch 96/200
8000/8000 [==============================] - 1s 73us/step - loss: 0.3305 - acc: 0.8662
Epoch 97/200
8000/8000 [==============================] - 1s 74us/step - loss: 0.3295 - acc: 0.8664
Epoch 98/200
8000/8000 [==============================] - 1s 74us/step - loss: 0.3308 - acc: 0.8672
Epoch 99/200
8000/8000 [==============================] - 1s 73us/step - loss: 0.3325 - acc: 0.8661
Epoch 100/200
8000/8000 [==============================] - 1s 72us/step - loss: 0.3298 - acc: 0.8664
Epoch 101/20

8000/8000 [==============================] - 1s 79us/step - loss: 0.3269 - acc: 0.8680
Epoch 173/200
8000/8000 [==============================] - 1s 74us/step - loss: 0.3262 - acc: 0.8710
Epoch 174/200
8000/8000 [==============================] - 1s 76us/step - loss: 0.3284 - acc: 0.8681
Epoch 175/200
8000/8000 [==============================] - 1s 75us/step - loss: 0.3258 - acc: 0.8680
Epoch 176/200
8000/8000 [==============================] - 1s 75us/step - loss: 0.3261 - acc: 0.8670
Epoch 177/200
8000/8000 [==============================] - 1s 80us/step - loss: 0.3264 - acc: 0.8699
Epoch 178/200
8000/8000 [==============================] - 1s 89us/step - loss: 0.3255 - acc: 0.8696
Epoch 179/200
8000/8000 [==============================] - 1s 79us/step - loss: 0.3269 - acc: 0.8700
Epoch 180/200
8000/8000 [==============================] - 1s 82us/step - loss: 0.3257 - acc: 0.8681
Epoch 181/200
8000/8000 [==============================] - 1s 79us/step - loss: 0.3256 - acc: 0.8698
Epoc

8000/8000 [==============================] - 1s 138us/step - loss: 0.5062 - acc: 0.7960
Epoch 43/200
8000/8000 [==============================] - 1s 119us/step - loss: 0.5061 - acc: 0.7960
Epoch 44/200
8000/8000 [==============================] - 1s 117us/step - loss: 0.5061 - acc: 0.7960
Epoch 45/200
8000/8000 [==============================] - 1s 105us/step - loss: 0.5060 - acc: 0.7960
Epoch 46/200
8000/8000 [==============================] - 1s 120us/step - loss: 0.5060 - acc: 0.7960
Epoch 47/200
8000/8000 [==============================] - 1s 118us/step - loss: 0.5060 - acc: 0.7960
Epoch 48/200
8000/8000 [==============================] - 1s 95us/step - loss: 0.5060 - acc: 0.7960
Epoch 49/200
8000/8000 [==============================] - 1s 96us/step - loss: 0.5059 - acc: 0.7960
Epoch 50/200
8000/8000 [==============================] - 1s 100us/step - loss: 0.5059 - acc: 0.7960
Epoch 51/200
8000/8000 [==============================] - 1s 98us/step - loss: 0.5059 - acc: 0.7960
Epoch 

8000/8000 [==============================] - 1s 92us/step - loss: 0.5059 - acc: 0.7960
Epoch 124/200
8000/8000 [==============================] - 1s 89us/step - loss: 0.5059 - acc: 0.7960
Epoch 125/200
8000/8000 [==============================] - 1s 90us/step - loss: 0.5059 - acc: 0.7960
Epoch 126/200
8000/8000 [==============================] - 1s 89us/step - loss: 0.5059 - acc: 0.7960
Epoch 127/200
8000/8000 [==============================] - 1s 90us/step - loss: 0.5059 - acc: 0.7960
Epoch 128/200
8000/8000 [==============================] - 1s 96us/step - loss: 0.5059 - acc: 0.7960
Epoch 129/200
8000/8000 [==============================] - 1s 91us/step - loss: 0.5059 - acc: 0.7960
Epoch 130/200
8000/8000 [==============================] - 1s 93us/step - loss: 0.5059 - acc: 0.7960
Epoch 131/200
8000/8000 [==============================] - 1s 91us/step - loss: 0.5059 - acc: 0.7960
Epoch 132/200
8000/8000 [==============================] - 1s 92us/step - loss: 0.5059 - acc: 0.7960
Epoc

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)